In [ ]:
import glob
import json
import os
from multiprocessing import Pool, pool

import cv2
import numpy as np
import csv

import pandas as pd
from scipy.io import loadmat
import time
from matplotlib import pyplot as plt

<h1> Calibration Facilitator Notebook </h1>

<h3> Define directory structure for calibration images </h3>

In [ ]:
left_validation_dir = '/Users/aloksaxena/Documents/aquabyteai/repos/cv_research/alok/notebooks/calibration/data/blom_pen_1_enclosure/validation_images/left'
right_validation_dir = '/Users/aloksaxena/Documents/aquabyteai/repos/cv_research/alok/notebooks/calibration/data/blom_pen_1_enclosure/validation_images/right'
stereo_params_f = '/Users/aloksaxena/Documents/aquabyteai/repos/cv_research/alok/notebooks/calibration/data/blom_pen_1_enclosure/2019-04-26_blom_kjeppevikholmen_pen_1.json'


In [ ]:
def load_params(params_file):
    params = json.load(open(params_file))
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    imageSize = (4096, 3000)
    
    # perform rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)
    
    return left_maps, right_maps

In [ ]:
left_maps, right_maps = load_params(stereo_params_f)

In [ ]:
def remap_pair(left_img_path, right_img_path, left_maps, right_maps):
    
    img_left = cv2.imread(left_img_path)
    img_right = cv2.imread(right_img_path)
    remap_left = cv2.remap(img_left, left_maps[0], left_maps[1], cv2.INTER_LANCZOS4)
    remap_right = cv2.remap(img_right, right_maps[0], right_maps[1], cv2.INTER_LANCZOS4)
    return remap_left, remap_right

def find_corners(remap):
    gray = cv2.cvtColor(remap,cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (1024, 750))
    
    ret, resized_corners = cv2.findChessboardCorners(resized, (9,6),None)
    if ret:
        corners = resized_corners * 4
        adj_corners = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        return np.squeeze(adj_corners)
    else:
        return None

In [ ]:
image_to_checkerboard_points = {}
left_image_fs = glob.glob(os.path.join(left_validation_dir, 'left_*.jpg'))[:7]
for left_image_f in left_image_fs:
    right_image_f = left_image_f.replace('left', 'right')
    if not os.path.exists(right_image_f):
        print('Path does not exist: {}'.format(right_image_f))
        continue
    
    remap_left, remap_right = remap_pair(left_image_f, right_image_f, left_maps, right_maps)
    
    left_corners = find_corners(remap_left)
    right_corners = find_corners(remap_right)
    if left_corners is not None and right_corners is not None:
        image_to_checkerboard_points[left_image_f] = {}
        image_to_checkerboard_points[left_image_f]['left_corners'] = left_corners
        image_to_checkerboard_points[left_image_f]['right_corners'] = right_corners

<h1> Get predicted vs. ground truth distances </h1>

In [ ]:
# all distance are in meters
stereo_params = json.load(open(stereo_params_f))
FOCAL_LENGTH_PIXEL = stereo_params['CameraParameters1']['FocalLength'][0]
BASELINE = abs(stereo_params['TranslationOfCamera2'][0] / 1e3) # convert millimeters to meters and use absolute value
PIXEL_SIZE_M = 3.45 * 1e-6
FOCAL_LENGTH = FOCAL_LENGTH_PIXEL * PIXEL_SIZE_M
IMAGE_SENSOR_WIDTH = 0.01412
IMAGE_SENSOR_HEIGHT = 0.01035
PIXEL_COUNT_WIDTH = 4096
PIXEL_COUNT_HEIGHT = 3000
CHECKERBOARD_SIDE_LENGTH = 0.049294

In [ ]:
def convert_to_world_point(x, y, d):
    """ from pixel coordinates to world coordinates """
    
    image_center_x = PIXEL_COUNT_WIDTH / 2.0  
    image_center_y = PIXEL_COUNT_HEIGHT / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (IMAGE_SENSOR_WIDTH / 4096)
    sensor_z = px_z * (IMAGE_SENSOR_HEIGHT / 3000)

    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / FOCAL_LENGTH
    world_z = (world_y * sensor_z) / FOCAL_LENGTH
    return np.array([world_x, world_y, world_z])



def depth_from_disp(disp):
    """ calculate the depth of the point based on the disparity value """
    depth = FOCAL_LENGTH_PIXEL*BASELINE / np.array(disp)
    return depth

def euclidean_distance(p1, p2):
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)**0.5

def distance_between_positions(i, j):
    p1_ax_0_pos = i // 9
    p1_ax_1_pos = i % 9
    p2_ax_0_pos = j // 9
    p2_ax_1_pos = j % 9
    return CHECKERBOARD_SIDE_LENGTH * ((p1_ax_0_pos - p2_ax_0_pos)**2 + (p1_ax_1_pos - p2_ax_1_pos)**2)**0.5


In [ ]:
world_coordinate_matrix = np.empty([len(list(image_to_checkerboard_points.keys())), 54, 3])
for idx, left_image_f in enumerate(image_to_checkerboard_points.keys()):
    left_corners = image_to_checkerboard_points[left_image_f]['left_corners']
    right_corners = image_to_checkerboard_points[left_image_f]['right_corners']
    for i in range(42):
        disp = abs(left_corners[i][0] - right_corners[i][0])
        print(abs(left_corners[i][1] - right_corners[i][1]))
        depth = depth_from_disp(disp)
        world_coordinate_matrix[idx, i, :] = convert_to_world_point(left_corners[i][0], left_corners[i][1], depth)
        
        
    

In [ ]:
analysis_df = pd.DataFrame()
for idx in range(world_coordinate_matrix.shape[0]):
    for i in range(world_coordinate_matrix.shape[1]-1):
        for j in range(i+1, world_coordinate_matrix.shape[1]):
            p1 = world_coordinate_matrix[idx, i, :]
            p2 = world_coordinate_matrix[idx, j, :]
            predicted_distance = euclidean_distance(p1, p2)
            ground_truth_distance = distance_between_positions(i, j)
            row = {
                'predicted_distance': predicted_distance,
                'ground_truth_distance': ground_truth_distance
            }
            analysis_df = analysis_df.append(row, ignore_index=True)
    
analysis_df['deviation'] = analysis_df['predicted_distance'] - analysis_df['ground_truth_distance']
analysis_df['pct_deviation'] = analysis_df['deviation'] / analysis_df['ground_truth_distance']

In [ ]:
plt.figure(figsize=(12, 8))
plt.hist(analysis_df[analysis_df.ground_truth_distance > 0.4]['deviation'])
plt.xlabel('Predicted distance deviation (meters)')
plt.ylabel('Ground distance deviation (meters)')
plt.show()

In [ ]:
x2, y2, z2

In [ ]:
((x - x2)**2 + (y-y2)**2 + (z-z2)**2)**.5

In [ ]:
np.squeeze(left_corners)

In [ ]:
right_corners